In [59]:
import pandas as pd
import numpy as np
from scipy.interpolate import PchipInterpolator

from bokeh.plotting import figure, show
from bokeh.io import curdoc, output_notebook
from bokeh.models import BoxZoomTool, ResetTool, PanTool, WheelZoomTool, HoverTool, AnnularWedge, ColumnDataSource, Legend, LegendItem, Plot, Range1d
from bokeh.palettes import PuBu, Inferno256, Category20

In [60]:
df_purchase = pd.read_csv("./purchases2.csv")
df_cash = pd.read_csv("./encashments2.csv")

In [61]:
df_purchase.rename(columns={'Date of\rPurchase': 'Date', 'Name of the Purchaser': 'Entity'}, inplace=True)
df_purchase = df_purchase[df_purchase["Sr No."] != "Sr No."]
df_purchase = df_purchase.reset_index().drop(["index"], axis=1)
df_purchase

,Sr No.,Reference No (URN),Journal Date,Date,Date of Expiry,Entity,Prefix,Bond\rNumber,Denominations,Issue Branch Code,Issue Teller,Status
0,1,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,TL,11448,"10,00,000",00001,5899230,Paid
1,2,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,TL,11447,"10,00,000",00001,5899230,Paid
2,3,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,TL,11441,"10,00,000",00001,5899230,Paid
3,4,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,OL,1113,"1,00,000",00001,5899230,Paid
4,5,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,OL,1118,"1,00,000",00001,5899230,Paid
...,...,...,...,...,...,...,...,...,...,...,...,...
18866,18867,00001202401110000003729,11/Jan/2024,11/Jan/2024,25/Jan/2024,WESTWELL GASES PRIVATE LIMITED,OC,15695,"1,00,00,000",00001,5898161,Paid
18867,18868,00001202401110000003729,11/Jan/2024,11/Jan/2024,25/Jan/2024,WESTWELL GASES PRIVATE LIMITED,OC,15693,"1,00,00,000",00001,5898161,Paid
18868,18869,00001202401110000003729,11/Jan/2024,11/Jan/2024,25/Jan/2024,WESTWELL GASES PRIVATE LIMITED,OC,15697,"1,00,00,000",00001,5898161,Paid
18869,18870,00001202401110000003729,11/Jan/2024,11/Jan/2024,25/Jan/2024,WESTWELL GASES PRIVATE LIMITED,OC,17803,"1,00,00,000",00001,5898161,Paid


In [62]:
df_cash.rename(columns={'Date of\rEncashment': 'Date', 'Name of the Political Party': 'Entity'}, inplace=True)
df_cash = df_cash[df_cash["Sr No."] != "Sr No."]
df_cash = df_cash.reset_index().drop(["index"], axis=1)
df_cash

,Sr No.,Date,Entity,Account no. of\rPolitical Party,Prefix,Bond\rNumber,Denominations,Pay Branch\rCode,Pay Teller
0,1,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,775,"1,00,00,000",00800,2770121
1,2,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3975,"1,00,00,000",00800,2770121
2,3,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3967,"1,00,00,000",00800,2770121
3,4,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,10418,"10,00,000",00800,2770121
4,5,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,126,"10,00,000",00800,2770121
...,...,...,...,...,...,...,...,...,...
20416,20417,24/Jan/2024,JANASENA PARTY,*******3929,TL,2619,"10,00,000",00847,7620160
20417,20418,24/Jan/2024,JANASENA PARTY,*******3929,TL,2608,"10,00,000",00847,7620160
20418,20419,24/Jan/2024,JANASENA PARTY,*******3929,TL,2633,"10,00,000",00847,7620160
20419,20420,24/Jan/2024,JANASENA PARTY,*******3929,TL,2627,"10,00,000",00847,7620160


In [63]:
def create_timeseries(df):
    df["Entity"] = df["Entity"].apply(lambda x: x.replace("(POLITICAL PARTY)", "").strip())
    df["Denominations"] = df["Denominations"].apply(lambda x: float(x.replace(",", ""))/10000000)
    df["Date"] = pd.to_datetime(df["Date"]).apply(lambda x: int(x.strftime('%Y-%m').replace("-", "")))
    df = df.groupby(["Date", "Entity"]).agg({"Denominations": lambda x: x.sum()}).reset_index()
    df.sort_values(by="Date", inplace=True)
    df['Cumulative'] = df.groupby('Entity')['Denominations'].cumsum()
    return df

In [64]:
def create_aggregate(df):
    df["Entity"] = df["Entity"].apply(lambda x: x.replace("(POLITICAL PARTY)", "").strip())
    df = df.groupby(["Entity"]).agg({"Denominations": lambda x: x.sum()}).reset_index()
    df.sort_values(by="Denominations", ascending=False, inplace=True)
    return df

In [65]:
def plot_scatter(x, y, title='Scatter Plot of X and Y Coordinates', x_label='X', y_label='Y', color="cyan", color_map=None, size_map=None):
    output_notebook()
    curdoc().theme = 'dark_minimal'

    p = figure(width=800, height=600, title=title)

    use_color = color_map if color_map is not None else color
    use_size = size_map if size_map is not None else 8
    p.scatter(x, y, size=use_size, color=use_color, alpha=0.5, legend_label='Data')

    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.grid.visible = True

    coefficients = np.polyfit(x, y, 1)
    slope = coefficients[0]
    intercept = coefficients[1]
    trendline = slope * x + intercept
    p.line(x, trendline, line_color='blue', line_width=1, legend_label='Trendline')

    hover = HoverTool(tooltips=[('X', '@x'), ('Y', '@y')])
    p.add_tools(hover)
    p.add_tools(PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool())
    show(p)

In [66]:
def plot_lines(df, x, y, cat, title='Line Chart', x_label='X', y_label='Y', color_map=None):
    output_notebook()
    curdoc().theme = 'dark_minimal'

    p = figure(width=1200, height=600, title=title)

    aggregated_df = df.groupby(cat)[y].sum().reset_index()
    top_20_categories = aggregated_df.nlargest(20, y)[cat]
    df = df[df[cat].isin(top_20_categories)]

    categories = df[cat].unique()
    colors = color_map if color_map else Category20[len(categories)]

    for i, category in enumerate(categories):
        category_data = df[df[cat] == category]
        p.line(category_data[x], category_data[y], line_color=colors[i % len(colors)], line_width=2, legend_label=category)

    p.legend.spacing = 0
    p.legend.margin = 0
    p.legend.padding = 0
    p.legend.label_text_font_size = '6pt'
    p.legend.click_policy = 'hide'
    p.add_layout(p.legend[0], 'right')

    p.add_tools(PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool())
    show(p)

In [67]:
df_purchase_time = create_timeseries(df_purchase)
df_purchase_time

,Date,Entity,Denominations,Cumulative
0,201904,A B C INDIA LIMITED,0.4000,0.4000
91,201904,RAIPUR BOTLING COMPANY,1.0001,1.0001
92,201904,RAJEEV KUMAR JAIN,0.2000,0.2000
94,201904,RENUKA INVESTMENTS AND FINANCE LTD,5.0000,5.0000
95,201904,RENUKESHWAR INVESTMENTS PVT LTD,5.0000,5.0000
...,...,...,...,...
1795,202401,GVPR ENGINEERS LTD,5.0000,10.0000
1794,202401,GRANULES INDIA LIMITED,3.0000,3.0000
1793,202401,GK ENERGY MARKETERS PRIVATE LIMITED,6.0000,6.0000
1801,202401,JAINENDRA PRAVINCHANDRA SHAH,0.9000,0.9000


In [68]:
df_cash_time = create_timeseries(df_cash)
df_cash_time

,Date,Entity,Denominations,Cumulative
0,201904,AAM AADMI PARTY,0.20,0.2000
17,201904,YSR CONGRESS PARTY (YUVAJANA SRAMIKA RYTHU C...,8.25,8.2500
16,201904,TELUGU DESAM PARTY,7.30,7.3000
15,201904,SHIVSENA,14.63,14.6300
14,201904,SHIROMANI AKALI DAL,6.26,6.2600
...,...,...,...,...
179,202401,BHARATIYA JANATA PARTY,202.00,6060.5111
178,202401,ALL INDIA TRINAMOOL CONGRESS,130.45,1609.5314
187,202401,TELUGU DESAM PARTY,118.20,218.8800
182,202401,"PRESIDENT, ALL INDIA CONGRESS COMMITTEE",35.90,1421.8655


In [69]:
df_purchase_agg = create_aggregate(df_purchase)
df_purchase_agg

,Entity,Denominations
356,FUTURE GAMING AND HOTEL SERVICES PR,1208.0000
654,MEGHA ENGINEERING AND INFRASTRUCTURES LI MITED,821.0000
868,QWIKSUPPLYCHAINPRIVATELIMITED,410.0000
419,HALDIA ENERGY LIMITED,377.0000
1252,VEDANTA LIMITED,375.6500
...,...,...
73,ANKUR SINGHAL S O SH ANIL SIN,0.0001
272,DAMINI NATH,0.0001
696,MR. SAMEER BHATIA,0.0001
88,ARAVIND S,0.0001


In [70]:
df_cash_agg = create_aggregate(df_cash)
df_cash_agg

,Entity,Denominations
5,BHARATIYA JANATA PARTY,6060.5111
3,ALL INDIA TRINAMOOL CONGRESS,1609.5314
16,"PRESIDENT, ALL INDIA CONGRESS COMMITTEE",1421.8655
4,BHARAT RASHTRA SAMITHI,1214.7099
7,BIJU JANATA DAL,775.5000
8,DRAVIDA MUNNETRA KAZHAGAM (DMK),639.0000
23,YSR CONGRESS PARTY (YUVAJANA SRAMIKA RYTHU C...,337.0000
22,TELUGU DESAM PARTY,218.8800
19,SHIVSENA,159.3814
17,RASHTRIYA JANTA DAL,73.5000


In [71]:
df_purchase_time.to_csv("./df_purchase_time.csv", index=False)
df_cash_time.to_csv("./df_cash_time.csv", index=False)
df_purchase_agg.to_csv("./df_purchase_agg.csv", index=False)
df_cash_agg.to_csv("./df_cash_agg.csv", index=False)

In [72]:
color_map = [
    "#7F00FF",  # Purple
    "#FF00FF",  # Magenta
    "#FF0000",  # Red
    "#FF8C00",  # Dark Orange
    "#00FFFF",  # Cyan
    "#F5F5DC",  # Beige
    "#0000FF",  # Blue
    "#C0C0C0",  # Silver
    "#808080",  # Gray
    "#FAEBD7",  # Antique White
    "#00FFFF",  # Aqua
    "#FFE4C4",  # Bisque
    "#F0F8FF",  # Alice Blue
    "#FFA500",  # Orange
    "#F0FFFF",  # Azure
    "#008000",  # Green
    "#FFEBCD",  # Blanched Almond
    "#7FFFD4",  # Aquamarine
    "#8A2BE2",  # Blue Violet
    "#A52A2A",  # Brown
    "#DEB887",  # Burly Wood
    "#5F9EA0",  # Cadet Blue
    "#7FFF00",  # Chartreuse
    "#D2691E",  # Chocolate
]

In [73]:
plot_lines(df_cash_time, "Date", "Denominations", "Entity", color_map=color_map, title="Annual Encashments by Party")

Loading BokehJS ...

In [74]:
plot_lines(df_cash_time, "Date", "Cumulative", "Entity", color_map=color_map, title="Cumulative Encashments by Party")

Loading BokehJS ...

In [75]:
plot_lines(df_purchase_time, "Date", "Denominations", "Entity", color_map=color_map, title="Annual Donations by Entity")

Loading BokehJS ...

In [76]:
plot_lines(df_purchase_time, "Date", "Cumulative", "Entity", color_map=color_map, title="Cumulative Donations by Entity")

Loading BokehJS ...